In [45]:
import captacity
from pydub import AudioSegment
import subprocess
import numpy as np
import captacity
import json
import math
import cv2
import os
import sys

## Test Code on Generating Segments from Audio

In [46]:
def create_segments(narrations, output_dir):
    segments = []

    offset = 0
    for i, narration in enumerate(narrations):
        audio_file = os.path.join(output_dir, "narrations", f"narration_{i+1}.mp3")

        try:
            t_segments = captacity.transcriber.transcribe_locally(
                audio_file=audio_file,
                prompt=narration,
            )
        except ImportError:
            t_segments = captacity.transcriber.transcribe_with_api(
                audio_file=audio_file,
                prompt=narration,
            )

        o_segments = offset_segments(t_segments, offset)

        segments += o_segments
        offset += get_audio_duration(audio_file) / 1000

    return segments

def offset_segments(segments: list[dict], offset: float):
    for segment in segments:
        segment["start"] += offset
        segment["end"] += offset
        for word in segment["words"]:
            word["start"] += offset
            word["end"] += offset
    return segments

def get_audio_duration(audio_file):
    """Get the duration of a single audio file."""
    return len(AudioSegment.from_file(audio_file))


In [92]:
import json

# Output from LLM parsed into JSON containing text/image keys per slide
json_file_path = '/Users/vivektrivedy/Documents/shortrocity/shorts/1721704640/data.json'
with open(json_file_path, 'r') as file:
    data_dict = json.load(file)

# just get the text descriptions - ie the stuff we want to narrate with Eleven Labs
narrations = [seg['content'] if seg['type'] == 'text' else None for seg in data_dict]
narrations = [narration.strip('"') for narration in narrations if narration is not None] 

output_dir = '/Users/vivektrivedy/Documents/shortrocity/shorts/1721704640'
segments = []

offset = 0
for i, narration in enumerate(narrations):
    audio_file = os.path.join(output_dir, "narrations", f"narration_{i+1}.mp3")

    try:
        t_segments = captacity.transcriber.transcribe_locally(
            audio_file=audio_file,
            prompt=narration,
        )
    except ImportError:
        t_segments = captacity.transcriber.transcribe_with_api(
            audio_file=audio_file,
            prompt=narration,
        )

    o_segments = offset_segments(t_segments, offset)

    segments += o_segments
    offset += get_audio_duration(audio_file) / 1000

In [129]:
t_segments

[{'id': 0,
  'seek': 0,
  'start': np.float64(14.207999999999998),
  'end': np.float64(17.407999999999998),
  'text': ' Meet the twilight zone—a hidden world teeming with life!',
  'tokens': [50364,
   22963,
   264,
   683,
   27797,
   6668,
   2958,
   64,
   7633,
   1002,
   535,
   443,
   278,
   365,
   993,
   0,
   50533],
  'temperature': 0.0,
  'avg_logprob': -0.1664121945699056,
  'compression_ratio': 0.8923076923076924,
  'no_speech_prob': 0.029924781993031502,
  'words': [{'word': ' Meet',
    'start': np.float64(14.207999999999998),
    'end': np.float64(14.427999999999999),
    'probability': np.float64(0.9195890426635742)},
   {'word': ' the',
    'start': np.float64(14.427999999999999),
    'end': np.float64(14.607999999999999),
    'probability': np.float64(0.8060223460197449)},
   {'word': ' twilight',
    'start': np.float64(14.607999999999999),
    'end': np.float64(15.027999999999999),
    'probability': np.float64(0.6087907031178474)},
   {'word': ' zone—a',
  

### Test with Segments
See if replacing this with word level stamps from Eleven Labs also works!

In [179]:
import copy
output = audio_word_parser(response_dict)
output2 = copy.deepcopy(output)
output3 = copy.deepcopy(output)
combined_outputs = [output, output2, output3]
combined_outputs

[{'start': np.float64(0.0),
  'end': np.float64(2.368),
  'words': [{'word': 'Ever',
    'start': np.float64(0.0),
    'end': np.float64(0.337)},
   {'word': 'wondered', 'start': np.float64(0.383), 'end': np.float64(0.685)},
   {'word': 'how', 'start': np.float64(0.72), 'end': np.float64(0.824)},
   {'word': 'we', 'start': np.float64(0.871), 'end': np.float64(0.952)},
   {'word': 'cracked', 'start': np.float64(1.01), 'end': np.float64(1.3)},
   {'word': 'the', 'start': np.float64(1.347), 'end': np.float64(1.416)},
   {'word': 'code', 'start': np.float64(1.463), 'end': np.float64(1.66)},
   {'word': 'of', 'start': np.float64(1.683), 'end': np.float64(1.73)},
   {'word': 'life?', 'start': np.float64(1.8), 'end': np.float64(2.368)}]},
 {'start': np.float64(0.0),
  'end': np.float64(2.368),
  'words': [{'word': 'Ever',
    'start': np.float64(0.0),
    'end': np.float64(0.337)},
   {'word': 'wondered', 'start': np.float64(0.383), 'end': np.float64(0.685)},
   {'word': 'how', 'start': np.fl

In [180]:
new_segs = []
offset = 0
for ix, seg in enumerate(combined_outputs):
    seg = [seg]
    o_segments = offset_segments(seg, offset)
    new_segs += o_segments
    offset += seg[0]['end']

In [181]:
output_filename = '/Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/segment_test.mp4'
with_narration = "/Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/with_narration.mp4"
output_dir = '/Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/'

output_path = os.path.join(output_dir, output_filename)
input_path = os.path.join(output_dir, with_narration)

captacity.add_captions(
    video_file=input_path,
    output_file=output_path,
    segments=new_segs,
    print_info=True,
)

Extracting audio...
Generating video elements...
NOTICE: Word 'Everwonderedhowwe' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthe' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwondered' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwe' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwecrackedthe' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwecrackedthecodeoflife?Everwondered' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwe' is too long for the frame!
NOTICE: Word 'Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwecrackedthecodeoflife?Everwonderedhowwecrackedthe' is too long for the frame!


Generated in 00:01 (31 clips)
Rendering video...
Moviepy - Building video /Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/segment_test.mp4.
MoviePy - Writing audio in segment_testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/segment_test.mp4



Moviepy - Done !
Moviepy - video ready /Users/vivektrivedy/Documents/shortrocity/shorts/1721886782/segment_test.mp4
Generated in 00:01
Rendered in 01:24
Done in 01:25


## Use Eleven Labs to Directly get Word Level Timestamps

In [28]:
import requests

url = "https://api.elevenlabs.io/v1/voices"

response = requests.request("GET", url)
#pretty print the response
print(json.dumps(response.json(), indent=4))


{
    "voices": [
        {
            "voice_id": "EXAVITQu4vr4xnSDxMaL",
            "name": "Sarah",
            "samples": null,
            "category": "premade",
            "fine_tuning": {
                "is_allowed_to_fine_tune": true,
                "state": {},
                "verification_failures": [],
                "verification_attempts_count": 0,
                "manual_verification_requested": false,
                "language": "en",
                "progress": {},
                "message": {},
                "dataset_duration_seconds": null,
                "verification_attempts": null,
                "slice_ids": null,
                "manual_verification": null
            },
            "labels": {
                "age": "young",
                "accent": "american",
                "gender": "female",
                "use_case": "news",
                "description": "soft"
            },
            "description": null,
            "preview_url": "https

In [30]:
# The 'requests' and 'json' libraries are imported.
# 'requests' is used to send HTTP requests, while 'json' is used for parsing the JSON data that we receive from the API.
import requests
import json
import os

url = "https://api.elevenlabs.io/v1/voices"
response = requests.request("GET", url).json()

#pretty print the response
# print(json.dumps(response.json(), indent=4))

# A loop is created to iterate over each 'voice' in the 'voices' list from the parsed data.
# The 'voices' list consists of dictionaries, each representing a unique voice provided by the API.
for voice in response['voices']:
    # For each 'voice', the 'name' and 'voice_id' are printed out.
    # These keys in the voice dictionary contain values that provide information about the specific voice.
    print(f"{voice['name']}; {voice['voice_id']}")

Sarah; EXAVITQu4vr4xnSDxMaL
Laura; FGY2WhTYpPnrIDTdsKH5
Charlie; IKne3meq5aSn9XLyUdCD
George; JBFqnCBsd6RMkjVDRZzb
Callum; N2lVS1w4EtoT3dr4eOWO
Liam; TX3LPaxmHKxFdv7VOQHJ
Charlotte; XB0fDUnXU5powFXDhCwa
Alice; Xb7hH8MSUJpSbSDYk0k2
Matilda; XrExE9yKIg1WjnnlVkGX
Will; bIHbv24MWmeRgasZH58o
Jessica; cgSgspJ2msm6clMCkdW9
Eric; cjVigY5qzO86Huf0OWal
Chris; iP95p4xoKVk53GoZ742B
Brian; nPczCjzI2devNBz1zQrb
Daniel; onwK4e9ZLuTAKqWW03F9
Lily; pFZP5JQG7iQjIQuC4Bku
Bill; pqHfZKP75CvOlQylNhV4


In [102]:
import requests
import json
import base64
import os

data_test = ['hey sup', 'yo man', 'ok cmon']

for ix, text in enumerate(data_test):
    out = eleven_labs_api_call(ix, text)
    
def eleven_labs_api_call(ix,text):
    VOICE_ID = "21m00Tcm4TlvDq8ikWAM"  # Rachel
    eleven_api_key = os.environ['ELEVEN_API_KEY']
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}/with-timestamps"
    headers = {
    "Content-Type": "application/json",
    "xi-api-key": eleven_api_key
    }
    data = {
    "text": "Ever wondered how we cracked the code of life?",
    "model_id": "eleven_multilingual_v2",
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.75
    }
    }
    response = requests.post(
        url,
        json=data,
        headers=headers,
    )
    if response.status_code != 200:
        print(f"Error encountered, status: {response.status_code}, "f"content: {response.text}")
        quit()

    # convert the response which contains bytes into a JSON string from utf-8 encoding
    json_string = response.content.decode("utf-8")
    # parse the JSON string and load the data as a dictionary
    response_dict = json.loads(json_string)

    # the "audio_base64" entry in the dictionary contains the audio as a base64 encoded string, 
    # we need to decode it into bytes in order to save the audio as a file
    audio_bytes = base64.b64decode(response_dict["audio_base64"])

    with open(f'output_{ix}.mp3', 'wb') as f:
        f.write(audio_bytes)

    return response_dict


{'characters': ['E', 'v', 'e', 'r', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'e', 'd', ' ', 'h', 'o', 'w', ' ', 'w', 'e', ' ', 'c', 'r', 'a', 'c', 'k', 'e', 'd', ' ', 't', 'h', 'e', ' ', 'c', 'o', 'd', 'e', ' ', 'o', 'f', ' ', 'l', 'i', 'f', 'e', '?'], 'character_start_times_seconds': [0.0, 0.232, 0.279, 0.313, 0.337, 0.383, 0.418, 0.464, 0.499, 0.546, 0.58, 0.615, 0.662, 0.685, 0.72, 0.755, 0.789, 0.824, 0.871, 0.906, 0.952, 1.01, 1.057, 1.103, 1.149, 1.196, 1.231, 1.277, 1.3, 1.347, 1.37, 1.393, 1.416, 1.463, 1.509, 1.591, 1.625, 1.66, 1.683, 1.707, 1.73, 1.8, 1.846, 1.927, 1.974, 2.125], 'character_end_times_seconds': [0.232, 0.279, 0.313, 0.337, 0.383, 0.418, 0.464, 0.499, 0.546, 0.58, 0.615, 0.662, 0.685, 0.72, 0.755, 0.789, 0.824, 0.871, 0.906, 0.952, 1.01, 1.057, 1.103, 1.149, 1.196, 1.231, 1.277, 1.3, 1.347, 1.37, 1.393, 1.416, 1.463, 1.509, 1.591, 1.625, 1.66, 1.683, 1.707, 1.73, 1.8, 1.846, 1.927, 1.974, 2.125, 2.368]}


In [134]:
import numpy
def audio_word_parser(response_dict):
    """Convert character level timestamps from eleven labs into word level timestamps"""
    characters = response_dict['alignment']['characters']
    char_start_times = response_dict['alignment']['character_start_times_seconds']
    char_end_times = response_dict['alignment']['character_end_times_seconds']

    words = []
    current_word = ""
    current_word_start = np.float64(0)
    
    for i, char in enumerate(characters):
        if char != ' ':
            if not current_word:
                current_word_start = char_start_times[i]
            current_word += char
        
        if char == ' ' or i == len(characters) - 1:
            if current_word:
                words.append({
                    "word": current_word,
                    "start": np.float64(current_word_start),
                    "end": np.float64(char_end_times[i-1]) if char == ' ' else np.float64(char_end_times[i])
                })
                current_word = ""

    return {
        "start": np.float64(0.00),
        "end": np.float64(char_end_times[-1]),
        "words": words
    }


In [147]:
output = audio_word_parser(response_dict)

{'start': np.float64(0.0),
 'end': np.float64(2.368),
 'words': [{'word': 'Ever',
   'start': np.float64(0.0),
   'end': np.float64(0.337)},
  {'word': 'wondered', 'start': np.float64(0.383), 'end': np.float64(0.685)},
  {'word': 'how', 'start': np.float64(0.72), 'end': np.float64(0.824)},
  {'word': 'we', 'start': np.float64(0.871), 'end': np.float64(0.952)},
  {'word': 'cracked', 'start': np.float64(1.01), 'end': np.float64(1.3)},
  {'word': 'the', 'start': np.float64(1.347), 'end': np.float64(1.416)},
  {'word': 'code', 'start': np.float64(1.463), 'end': np.float64(1.66)},
  {'word': 'of', 'start': np.float64(1.683), 'end': np.float64(1.73)},
  {'word': 'life?', 'start': np.float64(1.8), 'end': np.float64(2.368)}]}

In [90]:
len(word_alignment['words']), len(word_alignment['word_start_times_seconds']), len(word_alignment['word_end_times_seconds'])

(21, 21, 21)

## Web Scraping Exploration

In [38]:
#pip install beautifulsoup
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 5.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [182]:
import requests
from bs4 import BeautifulSoup

def scrape_all_text(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract all text from the page
    text = soup.get_text(separator=' ', strip=True)
    
    return text

# Example usage
url = 'https://www.morningbrew.com/daily/stories/2024/06/23/are-soccer-stars-playing-too-much'
url2 = 'https://nutmegsoccer.beehiiv.com/p/match-day-27-28-recap'
newsletter_text = scrape_all_text(url2)
newsletter_text

# At this point, you would send newsletter_text to your LLM for summarization

'Attention Required! | Cloudflare Please enable cookies. Sorry, you have been blocked You are unable to access beehiiv.com Why have I been blocked? This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data. What can I do to resolve this? You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page. Cloudflare Ray ID: 8a914c6c6b7932c5 • Your IP: Click to reveal 100.19.46.146 • Performance & security by Cloudflare'

In [43]:
print(newsletter_text)

Attention Required! | Cloudflare Please enable cookies. Sorry, you have been blocked You are unable to access beehiiv.com Why have I been blocked? This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data. What can I do to resolve this? You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page. Cloudflare Ray ID: 8a89760a5e954cac • Your IP: Click to reveal 100.19.46.146 • Performance & security by Cloudflare
